In [1]:
%load_ext autoreload

In [ ]:
"""
Grandfather passage sentences = 
0: You wish to know about my grandfather 
1: Well he is nearly ninety three years old 
2: yet he still thinks as swiftly as ever
3: He dresses himself in an old black frock coat
4: usually several buttons missing
5: A long beard clings to his chin
6: giving those who observe him
7: a pronounced feeling of the utmost respect
8: When he speaks his voice is
9: just a bit cracked and quivers a bit
10: Twice each day he plays skillfully
11: and with zest upon a small organ
12: Except in the winter when
13: the snow or ice prevents
14: he slowly takes a short walk in
15: the open air each day
16: We have often urged him to walk
17: more and smoke less
18: but he always answers
19: Banana oil Grandfather likes 
20: to be modern in his language

Each sentence is articulated silently as one would naturally speak in English language.  
DATA is an array of (number snetences * 6, 22 - number channels, 20.000 - number time steps).
"""

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from spdLearning import spdNN
from spdLearning import optimizers 
from spdLearning import trainTest
from spdLearning import spdNet

In [3]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [4]:
dev = "cpu" 
device = torch.device(dev)

In [5]:
numberSentences = 21
trialsPerSentence = 6
numberTrials = numberSentences * trialsPerSentence
numberChannels = 22
windowLength = 5000 * 4

In [6]:
subjectNumber = 1
subject = "Subject" + str(subjectNumber)

In [7]:
DATA = np.load("Experiment2/" + subject + "/grandfatherPassageContinuousSentences.npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
DATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)

In [8]:
labelsBySentence = np.array([[i] * trialsPerSentence for i in range(numberSentences)]).reshape(numberTrials)

Indices =  {}
for i in range(numberSentences):
    Indices[i] = []
for i in range(len(labelsBySentence)):
    Indices[labelsBySentence[i]].append(i)

covariancesLabels = np.zeros((numberSentences, trialsPerSentence, numberChannels, numberChannels))
for i in range(numberSentences):
    for j in range(trialsPerSentence):
        covariancesLabels[i, j] = 1/windowLength * ((DATA[Indices[i][j]] @ DATA[Indices[i][j]].T))

In [9]:
trainFeatures = np.zeros((numberSentences * 4, numberChannels, numberChannels))
trainLabels = np.zeros((numberSentences * 4))
count = 0
for i in range(numberSentences):
    trainFeatures[count:count + 4] = covariancesLabels[i, :4]
    trainLabels[count:count + 4] = [i] * 4
    count += 4

testFeatures = np.zeros((numberSentences * 2, numberChannels, numberChannels))
testLabels = np.zeros((numberSentences * 2))
count = 0
for i in range(numberSentences):
    testFeatures[count:count + 2] = covariancesLabels[i, 4:]
    testLabels[count:count + 2] = [i] * 2
    count += 2

In [10]:
trainDataset = BaseDataset(trainFeatures, trainLabels)
testDataset = BaseDataset(testFeatures, testLabels)
trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

In [11]:
numberEpochs = 1000

model = spdNet.learnSPDMatrices(numberSentences).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)

lossFunction = nn.CrossEntropyLoss()
spdOptimizer = optimizers.MixOptimizer(model.parameters(), lr = 0.05)

6641


In [12]:
maxValue = 0
for epoch in range(numberEpochs):
    trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, spdOptimizer, lossFunction)
    testLoss, testAccuracy = trainTest.testOperation(model, device, testDataloader, lossFunction)
    if maxValue < testAccuracy:
        maxValue = testAccuracy
    print(f'Epoch: {epoch + 1}/{numberEpochs}, Training loss: {trainLoss:.4f}, Training accuracy: {trainAccuracy:.2f}%, Val loss: {testLoss:.4f}, Val accuracy: {testAccuracy:.2f}%')
print(maxValue)

Epoch: 1/1000, Training loss: 0.1112, Training accuracy: 4.76%, Val loss: 0.1480, Val accuracy: 7.14%
Epoch: 2/1000, Training loss: 0.1106, Training accuracy: 7.14%, Val loss: 0.1467, Val accuracy: 0.00%
Epoch: 3/1000, Training loss: 0.1096, Training accuracy: 4.76%, Val loss: 0.1464, Val accuracy: 2.38%
Epoch: 4/1000, Training loss: 0.1095, Training accuracy: 4.76%, Val loss: 0.1461, Val accuracy: 4.76%
Epoch: 5/1000, Training loss: 0.1092, Training accuracy: 4.76%, Val loss: 0.1461, Val accuracy: 7.14%
Epoch: 6/1000, Training loss: 0.1091, Training accuracy: 7.14%, Val loss: 0.1457, Val accuracy: 4.76%
Epoch: 7/1000, Training loss: 0.1089, Training accuracy: 7.14%, Val loss: 0.1457, Val accuracy: 4.76%
Epoch: 8/1000, Training loss: 0.1087, Training accuracy: 3.57%, Val loss: 0.1454, Val accuracy: 2.38%
Epoch: 9/1000, Training loss: 0.1088, Training accuracy: 2.38%, Val loss: 0.1453, Val accuracy: 4.76%
Epoch: 10/1000, Training loss: 0.1084, Training accuracy: 3.57%, Val loss: 0.1452,